In [2]:
import random
import gymnasium as gym
from pprint import pprint
from catanatron import Color,RandomPlayer
from catanatron.players.weighted_random import WeightedRandomPlayer, DevCardRandomPlayer, DoNothingRandomPlayer
from catanatron.players.weighted_random import WeightedRandomPlayer
from catanatron_experimental.machine_learning.players.minimax import AlphaBetaPlayer

from catanatron.players.search import VictoryPointPlayer
from catanatron_gym.envs.catanatron_env import from_action_space
import gymnasium as gym
import numpy as np
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.ppo_mask import MaskablePPO

from stable_baselines3 import PPO
from catanatron_gym.features import create_sample_vector, get_feature_ordering,create_sample
from catanatron_gym.envs.catanatron_env import from_action_space
# FEATURES = get_feature_ordering
from pprint import pprint
import random
import os
# os.environ["POSTGRES_USER"]="catanatron"
# os.environ["POSTGRES_PASSWORD"]="victorypoint"
# os.environ["POSTGRES_DB"]="catanatron_db"


In [10]:
def mask_fn(env) -> np.ndarray:
    valid_actions = env.get_valid_actions()
    mask = np.zeros(env.action_space.n, dtype=np.float32)
    mask[valid_actions] = 1

    return np.array([bool(i) for i in mask])

env = gym.make("catanatron_gym:catanatron-v1")
env = ActionMasker(env=env,action_mask_fn=mask_fn)
observation, info = env.reset()

model = MaskablePPO.load(path= "../logs/Catan_Baseline_rewards_Test/exp_001/latest_model_1000000_steps.zip",env=env)

# pprint(vars(env))
# print(observation)
for _ in range(1000):
    # your agent here (this takes random actions)
    action = model.predict(observation=observation,action_masks=mask_fn(env),deterministic=True)
    # print(action[0])
    observation, reward, terminated, truncated, info = env.step(action[0])
    done = terminated or truncated
    if done:
        # pprint(vars(env))
        # print(env.game.state)
        print(env.game.winning_color())
        observation, info = env.reset()
print(info)
env.close()

None
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/alex/anaconda3/envs/catanatron/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.get_valid_actions to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_valid_actions` for environment variables or `env.get_wrapper_attr('get_valid_actions')` that will search the reminding wrappers.
  logger.warn(
/home/alex/anaconda3/envs/catanatron/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.game to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.game` for environment variables or `env.get_wrapper_attr('game')` that will search the reminding wrappers.
  logger.warn(


Color.BLUE
Color.RED
Color.BLUE
Color.BLUE
Color.BLUE
{'valid_actions': [289, 147, 163]}


In [5]:
best_path = "../logs/Catan_Switch_Exp/exp_014/best_model.zip"
latest_path = "../logs/Catan_Switch_Exp/exp_004/latest_model_1000000_steps.zip"

env = gym.make("catanatron_gym:catanatron-switch-v1",config = {"enemies": [AlphaBetaPlayer(Color.RED)]})
observation, info = env.reset()
# pprint(vars(env.unwrapped))
# print(env.p0)
# print(observation)
model = PPO.load(path=best_path,env=env)
print(env.p0.color)
actions_list=[]
# print(model.policy)
for _ in range(100):
    # print(env.game.state.player_state)
    # your agent here (this takes random actions)
    # action = random.choice([0,1,2,3,4])
    action = model.predict(observation=observation,deterministic=True)
    # print(action[0].dtype)
    observation, reward, terminated, truncated, info = env.step(action[0])
    done = terminated or truncated
    env.render()
    if done:
        # pprint(vars(env))
        # print(done)
        # print(env.game.winning_color())
        # print(env.game.state.player_state)
        # print(info)
        actions_list.append(env.game.state.actions)
        observation, info = env.reset()
# print(info)
env.close()

running switch env
{'enemies': [AlphaBetaPlayer:RED(depth=2,value_fn=base_fn,prunning=False)]}
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Color.BLUE
in renderer


/home/alex/anaconda3/envs/catanatron/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.p0 to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.p0` for environment variables or `env.get_wrapper_attr('p0')` that will search the reminding wrappers.
  logger.warn(


set up driver


OperationalError: (psycopg2.OperationalError) connection to server at "127.0.0.1", port 5432 failed: FATAL:  could not open file "global/pg_filenode.map": Permission denied

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [13]:
print(str(actions_list[0][0][1]))

ActionType.BUILD_SETTLEMENT


In [ ]:
print(observation.shape)
print(info)
print(truncated)


(614,)
{'valid_actions': [289]}
False


In [ ]:
from catanatron import Color
from catanatron.players.weighted_random import WeightedRandomPlayer
from catanatron.players.search import VictoryPointPlayer
from catanatron_gym.envs.catanatron_env import from_action_space

def my_reward_function(game, p0_color):
    winning_color = game.winning_color()
    if p0_color == winning_color:
        return 100
    elif winning_color is None:
        return 0
    else:
        return -100

# 3-player catan on a "Mini" map (7 tiles) until 6 points.
env = gym.make(
    "catanatron_gym:catanatron-v1",
    config={
        "map_type": "BASE",
        "vps_to_win": 6,
        # "enemies": [Pla(Color.RED)],
        # "reward_function": my_reward_function,
        "representation": "vector",
    },
)                                                       
observation, info = env.reset()
# pprint(vars(env))
# print(observation.shape)
# env.p0 = VictoryPointPlayer(Color.BLUE)
pprint(vars(env.env))
agent = WeightedRandomPlayer(Color.WHITE)
env.p0 = agent
for _ in range(1000):
    # your agent here (this takes random actions)
    # valid_actions = env.unwrapped.get_valid_actions()# list[int]
    # print(len(valid_actions))
    # print(len(env.unwrapped.get_playable_actions()))
    # print(len(env.game.state.playable_actions))
    # print(from_action_space(94,env.game.state.playable_actions ))
    #convert valid actions to playable actions for agent
    # playable_actions = 
    # action = random.choice(env.unwrapped.get_valid_actions())
    # print(list(map(env.to_action_space, env.game.state.playable_actions)))
    # print(env.get_valid_actions())
    action =agent.decide(game = env.game, playable_actions=env.game.state.playable_actions)
    # print(action)
    observation, reward, terminated, truncated, info = env.step(action)
    # print(reward)
    # print(info)
    done = terminated or truncated
    if done:
        # pprint(vars(env))
        # print(env.game.state)
        # print(observation)
        print(reward)
        observation, info = env.reset()
print(env.enemies)
print(env.p0)
env.close()

{'_action_space': None,
 '_cached_spec': None,
 '_metadata': None,
 '_observation_space': None,
 '_reward_range': None,
 '_saved_kwargs': {},
 'checked_render': False,
 'checked_reset': True,
 'checked_step': False,
 'close_called': False,
 'env': <catanatron_gym.envs.catanatron_env.CatanatronEnv object at 0x7fa9554e9190>}
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
excep

In [ ]:
from catanatron import Game, RandomPlayer, Color
from catanatron.players.weighted_random import WeightedRandomPlayer
from catanatron.players.search import VictoryPointPlayer

# Play a simple 4v4 game
players = [
    RandomPlayer(Color.RED),
    WeightedRandomPlayer(Color.BLUE),
    RandomPlayer(Color.WHITE),
    RandomPlayer(Color.ORANGE),
]
game = Game(players)
print(game.play()) 

Color.ORANGE


In [ ]:
import gymnasium as gym
import numpy as np
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.ppo_mask import MaskablePPO

def mask_fn(env) -> np.ndarray:
    valid_actions = env.get_valid_actions()
    mask = np.zeros(env.action_space.n, dtype=np.float32)
    mask[valid_actions] = 1

    return np.array([bool(i) for i in mask])


# Init Environment and Model
env = gym.make("catanatron_gym:catanatron-v1")
env = ActionMasker(env, mask_fn)  # Wrap to enable masking
model = MaskablePPO(MaskableActorCriticPolicy, env, verbose=1)

# Train
model.learn(total_timesteps=1_000_000)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/alex/anaconda3/envs/catanatron/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.action_masks to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.action_masks` for environment variables or `env.get_wrapper_attr('action_masks')` that will search the reminding wrappers.
  logger.warn(
/home/alex/anaconda3/envs/catanatron/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.get_valid_actions to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_valid_actions` for environment variables or `env.get_wrapper_attr('get_valid_actions')` that will search the reminding wrappers.
  logger.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 364      |
|    ep_rew_mean     | -20      |
| time/              |          |
|    fps             | 555      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 483          |
|    ep_rew_mean          | -25          |
| time/                   |              |
|    fps                  | 494          |
|    iterations           | 2            |
|    time_elapsed         | 8            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0030232132 |
|    clip_fraction        | 0.0314       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.468       |
|    explained_variance   | 0.00386      |
|    learning_r

KeyboardInterrupt: 

In [3]:
from catanatron import Color
from catanatron.players.weighted_random import WeightedRandomPlayer
from catanatron.players.search import VictoryPointPlayer
from catanatron_gym.envs.catanatron_env import from_action_space
import gymnasium as gym
import numpy as np
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.ppo_mask import MaskablePPO

from stable_baselines3 import PPO

In [4]:


def mask_fn(env) -> np.ndarray:
    valid_actions = env.get_valid_actions()
    mask = np.zeros(env.action_space.n, dtype=np.float32)
    mask[valid_actions] = 1

    return np.array([bool(i) for i in mask])


# Init Environment and Model
env = gym.make("catanatron_gym:catanatron-switch-v1")
# env = ActionMasker(env, mask_fn)  # Wrap to enable masking
model = PPO("MlpPolicy", env, verbose=1)

# Train
model.learn(total_timesteps=1_000_000)

running switch env
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
2
[blue]Player:BLUE[/blue]
valid catan actions[Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=0), Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=1), Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=2), Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=3), Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=5), Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=6), Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=7), Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD

KeyboardInterrupt: 

In [2]:
from catanatron import Color,RandomPlayer
from catanatron.players.weighted_random import WeightedRandomPlayer, DevCardRandomPlayer, DoNothingRandomPlayer
from catanatron.players.weighted_random import WeightedRandomPlayer

from catanatron.players.search import VictoryPointPlayer
from catanatron_gym.envs.catanatron_env import from_action_space
import gymnasium as gym
import numpy as np
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.ppo_mask import MaskablePPO

from stable_baselines3 import PPO
from catanatron_gym.features import create_sample_vector, get_feature_ordering,create_sample
from catanatron_gym.envs.catanatron_env import from_action_space
# FEATURES = get_feature_ordering
from pprint import pprint
import random

In [ ]:
env = gym.make("catanatron_gym:catanatron-v1")
observation, info = env.reset()
FEATURES = get_feature_ordering(2)
pprint(vars(env))
# print(observation)
# agent = DevCardRandomPlayer(Color.BLUE)
# agent = VictoryPointPlayer(Color.BLUE)
# agent = RandomPlayer(Color.BLUE)
# env.p0 = WeightedRandomPlayer(Color.BLUE)

env.p0 = agent
# print((create_sample_vector(game =env.game,p0_color=env.p0.color)))
# print(len(env.game.state.playable_actions))
# print(env.game.state.playable_actions)
# print(len(env.unwrapped.get_valid_actions()))
# print(env.unwrapped.get_valid_actions())

# print(env.action_space)
print(agent.decide(game = env.game, playable_actions=env.unwrapped.game.state.playable_actions))
# print(observation)
for _ in range(1000):
    # your agent here (this takes random actions)
    # print(len(env.unwrapped.get_valid_actions()))
    # print(env.game.state.playable_actions)
    action = random.choice(env.unwrapped.get_valid_actions())
    observation, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    if done:
        # pprint(vars(env))
        # print(env.game.state)
        # p
        observation, info = env.reset()
print(info)
env.close()

{'_action_space': None,
 '_cached_spec': None,
 '_disable_render_order_enforcing': False,
 '_has_reset': True,
 '_metadata': None,
 '_observation_space': None,
 '_reward_range': None,
 '_saved_kwargs': {'disable_render_order_enforcing': False},
 'env': <PassiveEnvChecker<CatanatronEnv<catanatron-v1>>>}
bloadt actions [Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=0), Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=1), Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=2), Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=3), Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=4), Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=5), Action(color=<Color.BLUE: 'BLUE'

/home/alex/anaconda3/envs/catanatron/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.game to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.game` for environment variables or `env.get_wrapper_attr('game')` that will search the reminding wrappers.
  logger.warn(


{'valid_actions': [289]}


In [3]:
env = gym.make("catanatron_gym:catanatron-v1")
observation, info = env.reset()
# pprint(vars(env.unwrapped))
# print(observation)
# env.unwrapped.p0 = DoNothingRandomPlayer(color=Color.BLUE)
# env.unwrapped.p0 = DevCardRandomPlayer(color=Color.BLUE)
# env.unwrapped.p0 = RandomPlayer(color=Color.BLUE)
# pprint(vars(env.unwrapped))
pprint(env.unwrapped.p0)
for _ in range(1000):
    # your agent here (this takes random actions)
    action = random.choice(env.unwrapped.get_valid_actions())
    # print(env.unwrapped.get_valid_actions())
    print("playable actions\n",env.unwrapped.game.state.playable_actions)
    # print(action)
    
    # print(from_action_space(action, env.unwrapped.game.state.playable_actions))
    game = env.game
    print("agent",env.unwrapped.p0.decide(game = game ,playable_actions=env.unwrapped.game.state.playable_actions))
    # print(action)
    observation, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    if done:
        # pprint(vars(env))
        print(env.game.state)
        observation, info = env.reset()
print(info)
env.close()

None
Player:BLUE
playable actions
 [Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=0), Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=1), Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=2), Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=3), Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=4), Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=5), Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=6), Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=7), Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, va

/home/alex/anaconda3/envs/catanatron/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.game to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.game` for environment variables or `env.get_wrapper_attr('game')` that will search the reminding wrappers.
  logger.warn(


NotImplementedError: 